In [ ]:
import altair as alt
import pandas as pd

In [6]:
# Contar cuántas entradas tienen información en la columna 'DIRECTORES'
# Consideramos que una entrada tiene información si el valor no es nulo o vacío.
total_directores = df['DIRECTORES'].dropna().shape[0]

print(f"Total de entradas con información de directores: {total_directores}")

# Estimar el número de directoras mujeres basado en la proporción 1 de cada 10
# Usamos floor para obtener un número entero de directoras
mujeres_directoras_estimado = int(total_directores / 10)
hombres_directores_estimado = total_directores - mujeres_directoras_estimado

print(f"Estimado de directoras mujeres (basado en 1 de cada 10): {mujeres_directoras_estimado}")
print(f"Estimado de directores hombres: {hombres_directores_estimado}")

Total de entradas con información de directores: 254
Estimado de directoras mujeres (basado en 1 de cada 10): 25
Estimado de directores hombres: 229


In [38]:
# Crear un DataFrame para la visualización con las cifras actualizadas
data = {'Género': ['Hombres', 'Mujeres'],
        'Cantidad': [231, 17]} # Usamos las cifras proporcionadas por el usuario: 231 hombres y 17 mujeres
df_directores_estimado = pd.DataFrame(data)

display(df_directores_estimado)

Género  Cantidad
0  Hombres       231
1  Mujeres        17

In [4]:
import pandas as pd
df = pd.read_csv('/content/Base de datos final.csv')
display(df.head())

NOMBRE CANAL    AÑO  EPISODIOS  AUDIENCIA      DIRECTORES  \
0    Los días jóvenes     T13  1967         93  SIN INFO   Herval Rossano   
1      El loco estero     T13  1968  SIN INFO   SIN INFO       SIN INFO     
2        Incomunicados    T13  1968  SIN INFO   SIN INFO       SIN INFO     
3            El socio     T13  1968         68  SIN INFO      Helvio Soto   
4  El rosario de plata    T13  1969  SIN INFO   SIN INFO       SIN INFO     

                     REPARTO PRINCIPAL   Unnamed: 7  
0     Mirella Latorre y Mario Rodríguez         NaN  
1      Alicia Quiroga y Nelson Villagra         NaN  
2      Yoya Martínez y Jorge Sallorenzo         NaN  
3  Silvia Santelices y Leonardo Perucci         NaN  
4   Virginia Fischer y Leonardo Perucci         NaN

In [10]:
# Crear el gráfico de barras con Altair usando emojis
chart = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='middle',
    dx=0  # Ajuste horizontal si es necesario
).encode(
    x=alt.X('Género', axis=None), # No mostramos el eje x ya que el texto lo indica
    y=alt.Y('Cantidad', title='Cantidad de Teleseries'),
    text=alt.Text('Cantidad'), # Usamos la cantidad como texto inicialmente
    color='Género' # Coloreamos por género
).properties(
    title='Distribución de Teleseries por Género del Director'
)

# Ahora, vamos a superponer una capa con los emojis
emoji_layer = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='bottom', # Alineamos los emojis en la parte inferior de la "barra"
    fontSize=30 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('Género'),
    y=alt.Y('Cantidad'),
    text=alt.Text('Género',
                  scale=alt.Scale(
                      domain=['Hombres', 'Mujeres'],
                      range=['🧍‍♂️', '🧍‍♀️'] # Aquí cambias los emojis
                  )),
    color='Género'
)

# Combinar el gráfico de texto con los números y la capa de emojis
final_chart = chart + emoji_layer

final_chart.display()

SchemaValidationError: `Text` has no parameter named 'scale'

Existing parameter names are:
shorthand      bin         format       title   
aggregate      condition   formatType   type    
bandPosition   field       timeUnit             

See the help for `Text` to read the full description of these parameters

alt.LayerChart(...)

In [47]:
# Crear el gráfico de pictogramas con Altair
# Para posicionar los emojis de forma que parezcan "apilados" o en filas,
# necesitamos generar un índice para cada emoji dentro de su categoría de género.
# Primero, ordenamos por Género
df_expanded_sorted = df_expanded_sorted.sort_values('Género')

# Luego, añadimos un índice secuencial dentro de cada grupo de género
df_expanded_sorted['item_index'] = df_expanded_sorted.groupby('Género').cumcount()

# Definir el número de columnas antes de envolver
# Puedes ajustar este número para controlar cuántos emojis hay por fila antes de pasar a la siguiente
num_columns = 20 # Puedes cambiar este valor según cuántos emojis quieras por fila

df_expanded_sorted['col_index'] = df_expanded_sorted['item_index'] % num_columns
df_expanded_sorted['row_index'] = (df_expanded_sorted['item_index'] / num_columns).astype(int)


# Ahora creamos el gráfico usando el DataFrame ordenado y con el índice
chart = alt.Chart(df_expanded_sorted).mark_text(
    align='center',
    baseline='middle',
    fontSize=20 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('col_index:O', axis=None), # Usamos el índice de columna para la posición horizontal
    y=alt.Y('row_index:O', axis=None), # Usamos el índice de fila para la posición vertical dentro de cada grupo
    text=alt.condition(
        alt.datum.Género == 'Hombres',
        alt.value('👨🏻‍💻'),  # Emoji para Hombres
        alt.value('👩🏻‍💻')   # Emoji para Mujeres
    ),
    color=alt.Color('Género', legend=None), # Coloreamos por género y eliminamos la leyenda
    tooltip=['Género'] # Añadimos tooltip con el género
) # Eliminamos la propiedad title

# Para separar los grupos por género en columnas, usaremos la propiedad 'column'
final_chart = chart.facet(
    column=alt.Column('Género', title='Género del director según cada teleserie', header=alt.Header(
        titleOrient="top", # Cambiamos la orientación del título a arriba
        labelOrient="bottom",
        labelExpr="datum.value == 'Hombres' ? '🧍🏻‍♂️ Masculino' : '🧍🏻‍♀️ Femenino'" # Usamos labelExpr para mostrar emojis y texto
    ))
)


final_chart.display()

alt.FacetChart(...)

In [36]:
# Expandir el DataFrame df para tener una fila por cada teleserie con información de director
# Solo consideramos las filas donde la columna 'DIRECTORES' no sea nula y no sea 'SIN INFO'
df_directores_presentes = df[df['DIRECTORES'].dropna().str.strip() != 'SIN INFO'].copy()

# Para el propósito del tooltip, queremos asociar cada "pictograma" con una teleserie.
# Si estamos trabajando con la ESTIMACIÓN de 1 de cada 10, el vínculo directo con la teleserie original se pierde un poco.
# Sin embargo, podemos crear un DataFrame expandido que contenga las teleseries con director,
# y luego usar este DataFrame para la visualización, aplicando la lógica de género para el color/emoji.

# Si el objetivo es que el tooltip muestre el género ESTIMADO Y el nombre de la teleserie original,
# necesitamos que cada fila en df_expanded corresponda a una teleserie original con director.
# Luego, para la visualización, podemos asignar un género (H/M) basado en la proporción.

# Vamos a crear df_expanded con NOMBRE y DIRECTORES (para referencia, aunque no usemos DIRECTORES directamente para género)
# Y añadiremos una columna 'Género_Estimado' basada en la proporción, si eso es lo que se busca para el tooltip.

# Crear un DataFrame solo con las entradas que tienen director
df_con_directores = df[df['DIRECTORES'].notna() & (df['DIRECTORES'].str.strip() != 'SIN INFO')].copy()

# Reiniciar el índice para que sea más fácil trabajar
df_con_directores.reset_index(drop=True, inplace=True)

# Ahora, para el tooltip, queremos el nombre de la teleserie y el género.
# Si estamos trabajando con la estimación de 1 de cada 10, el género para el tooltip debería reflejar esa estimación por cada punto.
# Esto es complicado porque la estimación no se basa en la identidad real del director.

# Alternativa: Si el tooltip solo necesita mostrar el nombre de la teleserie y el GÉNERO ESTIMADO (H/M)
# para cada UNO de los pictogramas en el gráfico (que son 254 en total),
# podemos crear df_expanded_for_tooltip con 254 filas, cada una con un nombre de teleserie y un género estimado.
# Asignaremos los primeros 25 nombres de teleseries a 'Mujeres' y el resto a 'Hombres'.
# Esto ASUME que las primeras 25 teleseries con director en el DataFrame son dirigidas por mujeres según la ESTIMACIÓN.
# Esto no es preciso en la vida real, pero encaja con la lógica de la estimación 1 de cada 10 aplicada a los datos.

# Creamos una lista de nombres de teleseries con director
nombres_teleseries_con_director = df_con_directores['NOMBRE'].tolist()

# Creamos la lista de géneros estimados para cada teleserie (25 mujeres, 229 hombres)
generos_estimados_for_tooltip = ['Mujeres'] * mujeres_directoras_estimado + ['Hombres'] * hombres_directores_estimado

# Asegurarnos de que las listas tengan la misma longitud (deberían ser total_directores)
if len(nombres_teleseries_con_director) != len(generos_estimados_for_tooltip):
    # Si no coinciden, hay un problema con los cálculos o los datos.
    # Por ahora, usaremos la longitud menor para evitar errores, pero esto indica una inconsistencia.
    min_len = min(len(nombres_teleseries_con_director), len(generos_estimados_for_tooltip))
    nombres_teleseries_con_director = nombres_teleseries_con_director[:min_len]
    generos_estimados_for_tooltip = generos_estimados_for_tooltip[:min_len]


# Creamos el DataFrame expandido para el tooltip
df_expanded_for_tooltip = pd.DataFrame({
    'NOMBRE': nombres_teleseries_con_director,
    'Género_Estimado': generos_estimados_for_tooltip
})

# Este DataFrame df_expanded_for_tooltip tiene 254 filas, cada una representando una teleserie con director
# y su género estimado (según la proporción 1 de cada 10).
# Ahora necesitamos que df_expanded_sorted (usado para el gráfico) también tenga esta información.

# Vamos a REEMPLAZAR la creación de df_expanded_sorted
# Usaremos df_expanded_for_tooltip como base para el gráfico y añadiremos los índices de posición

df_expanded_sorted = df_expanded_for_tooltip.sort_values('Género_Estimado')

# Luego, añadimos un índice secuencial dentro de cada grupo de género estimado
df_expanded_sorted['item_index'] = df_expanded_sorted.groupby('Género_Estimado').cumcount()

# Definir el número de columnas antes de envolver (usamos el mismo num_columns de antes)
num_columns = 20 # Asegúrate de que esta variable esté definida en una celda anterior o aquí

df_expanded_sorted['col_index'] = df_expanded_sorted['item_index'] % num_columns
df_expanded_sorted['row_index'] = (df_expanded_sorted['item_index'] / num_columns).astype(int)

# Renombramos 'Género_Estimado' a 'Género' para que coincida con la codificación del gráfico existente
df_expanded_sorted.rename(columns={'Género_Estimado': 'Género'}, inplace=True)


display(df_expanded_sorted.head())
display(df_expanded_sorted.tail())

# Ahora que df_expanded_sorted tiene las columnas 'NOMBRE' y 'Género',
# podemos modificar la celda del gráfico (a0d6cf71) para incluir 'NOMBRE' en el tooltip.

/tmp/ipython-input-1499535845.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_directores_presentes = df[df['DIRECTORES'].dropna().str.strip() != 'SIN INFO'].copy()


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [34]:
# Guardar el gráfico como un archivo HTML
final_chart.save('genero_directores_pictograma.html')

In [39]:
# Expandir el DataFrame df_directores_estimado para tener una fila por cada teleserie
expanded_data = []
for index, row in df_directores_estimado.iterrows():
    genero = row['Género']
    cantidad = row['Cantidad']
    for i in range(cantidad):
        expanded_data.append({'Género': genero})

df_expanded = pd.DataFrame(expanded_data)

# Para posicionar los emojis de forma que parezcan "apilados" o en filas,
# necesitamos generar un índice para cada emoji dentro de su categoría de género.
# Primero, ordenamos por Género
df_expanded_sorted = df_expanded.sort_values('Género')

# Luego, añadimos un índice secuencial dentro de cada grupo de género
df_expanded_sorted['item_index'] = df_expanded_sorted.groupby('Género').cumcount()

# Definir el número de columnas antes de envolver (usamos el mismo num_columns de antes)
# Asegúrate de que num_columns esté definida en una celda anterior o aquí
if 'num_columns' not in locals():
    num_columns = 20 # Define un valor por defecto si no existe

df_expanded_sorted['col_index'] = df_expanded_sorted['item_index'] % num_columns
df_expanded_sorted['row_index'] = (df_expanded_sorted['item_index'] / num_columns).astype(int)

# Renombramos 'Género' a 'Género' (no cambia, pero mantenemos consistencia)
# df_expanded_sorted.rename(columns={'Género': 'Género'}, inplace=True) # Esta línea no es necesaria

display(df_expanded.head())
display(df_expanded.tail())
display(df_expanded_sorted.head())
display(df_expanded_sorted.tail())

Género
0  Hombres
1  Hombres
2  Hombres
3  Hombres
4  Hombres

Género
243  Mujeres
244  Mujeres
245  Mujeres
246  Mujeres
247  Mujeres

Género  item_index  col_index  row_index
0    Hombres           0          0          0
148  Hombres           1          1          0
149  Hombres           2          2          0
150  Hombres           3          3          0
151  Hombres           4          4          0

Género  item_index  col_index  row_index
233  Mujeres          12         12          0
231  Mujeres          13         13          0
246  Mujeres          14         14          0
238  Mujeres          15         15          0
247  Mujeres          16         16          0

In [12]:
# Crear el gráfico de barras con Altair usando emojis
chart = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='middle',
    dx=0  # Ajuste horizontal si es necesario
).encode(
    x=alt.X('Género', axis=None), # No mostramos el eje x ya que el texto lo indica
    y=alt.Y('Cantidad', title='Cantidad Estimada de Teleseries'),
    text=alt.Text('Cantidad'), # Usamos la cantidad como texto inicialmente
    color='Género' # Coloreamos por género
).properties(
    title='Estimación de Directores por Género (Basado en Proporción 1 de cada 10 mujeres)'
)

# Ahora, vamos a superponer una capa con los emojis
emoji_layer = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='bottom', # Alineamos los emojis en la parte inferior de la "barra"
    fontSize=30 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('Género'),
    y=alt.Y('Cantidad', stack=None), # Ensure emojis are not stacked
    text=alt.condition(
        alt.datum.Género == 'Hombres',
        alt.value('👨'),  # Emoji para Hombres
        alt.value('👩')   # Emoji para Mujeres
    ),
    color='Género'
)

# Combinar el gráfico de texto con los números y la capa de emojis
final_chart = chart + emoji_layer

final_chart.display()

alt.LayerChart(...)

# Task
Crea un gráfico de pictogramas con Altair usando el dataframe `df_directores_estimado` para comparar la cantidad de directores masculinos y femeninos. El gráfico debe mostrar 10 pictogramas por cada director masculino y 1 pictograma por cada directora femenina. Utiliza los emojis '👨🏻' para hombres y '👩🏻' para mujeres. El gráfico debe tener un título que diga "Género del director según cada teleserie", y el tooltip debe mostrar el género del director y el nombre de la teleserie. Guarda el gráfico como un archivo HTML llamado "distribucion_directores.html".

## Preparar datos (con nombre de teleserie)

### Subtask:
Modificar el código que crea `df_expanded` para incluir también el nombre de la teleserie para cada director.


**Reasoning**:
The subtask requires modifying the code that creates `df_expanded` to include the teleserie name and director information. The original `df_expanded` was created from `df_directores_estimado` which only contained aggregated counts. To include the teleserie names, we need to expand the original `df` based on the 'DIRECTORES' column, keeping rows where 'DIRECTORES' is not 'SIN INFO' or null.

